In [1]:
import os
from glob import glob
import numpy as np
import fitsio
from scipy.ndimage import gaussian_filter1d
from matplotlib import pyplot as plt
from time import time
from scipy.interpolate import interp1d
from astropy.io import fits

targetid_abs_g = np.load('/global/u1/z/zhaozhon/task/targetid_abs_g.npy')
targetid_abs_r = np.load('/global/u1/z/zhaozhon/task/targetid_abs_r.npy')
targetid_abs_i = np.load('/global/u1/z/zhaozhon/task/targetid_abs_i.npy')
z_g = np.load('/global/u1/z/zhaozhon/task/z_g.npy')
z_r = np.load('/global/u1/z/zhaozhon/task/z_r.npy')
z_i = np.load('/global/u1/z/zhaozhon/task/z_i.npy')
bp_g = np.load('/global/u1/z/zhaozhon/k-correction/Bandpass/bp_g.npy')
bp_r = np.load('/global/u1/z/zhaozhon/k-correction/Bandpass/bp_r.npy')
bp_i = np.load('/global/u1/z/zhaozhon/k-correction/Bandpass/bp_i.npy')

In [2]:
#zero_point
hdul = fits.open('/global/u1/z/zhaozhon/zero-point/bd_17d4708_stisnic_007.fits')
data = hdul[1].data
ws_z = data['WAVELENGTH']
fs_z = data['flux']
f_z = interp1d(ws_z,fs_z, kind='cubic')
ws_z_new = np.linspace(ws_z[0], ws_z[-1], 1000)
fs_z_new = f_z(ws_z_new)

In [3]:
ws_g = bp_g[0]
bp_g = bp_g[1]
f_bp_g = interp1d(ws_g, bp_g, kind='cubic')
ws_g_new = np.linspace(ws_g[0], ws_g[-1], 1000)
bp_g_new = f_bp_g(ws_g_new)

In [4]:
'''
# Plot the original data points
plt.plot(ws_g, bp_g,label='Data Points',color = 'black')

# Plot the cubic spline
plt.plot(ws_g_new, bp_g_new, label='Cubic Spline', color='red')

plt.legend()
plt.xlabel('wavelength(A)')
plt.ylabel('Transmission')
plt.title('Cubic Spline Interpolation')
plt.show()
'''

"\n# Plot the original data points\nplt.plot(ws_g, bp_g,label='Data Points',color = 'black')\n\n# Plot the cubic spline\nplt.plot(ws_g_new, bp_g_new, label='Cubic Spline', color='red')\n\nplt.legend()\nplt.xlabel('wavelength(A)')\nplt.ylabel('Transmission')\nplt.title('Cubic Spline Interpolation')\nplt.show()\n"

In [5]:
ws_r = bp_r[0]
bp_r = bp_r[1]
f_bp_r = interp1d(ws_r, bp_r, kind='cubic')
ws_r_new = np.linspace(ws_r[0], ws_r[-1], 1000)
bp_r_new = f_bp_r(ws_r_new)

In [6]:
'''
# Plot the original data points
plt.plot(ws_r, bp_r,label='Data Points',color = 'black')

# Plot the cubic spline
plt.plot(ws_r_new, bp_r_new, label='Cubic Spline', color='red')

plt.legend()
plt.xlabel('wavelength(A)')
plt.ylabel('Transmission')
plt.title('Cubic Spline Interpolation')
plt.show()
'''

"\n# Plot the original data points\nplt.plot(ws_r, bp_r,label='Data Points',color = 'black')\n\n# Plot the cubic spline\nplt.plot(ws_r_new, bp_r_new, label='Cubic Spline', color='red')\n\nplt.legend()\nplt.xlabel('wavelength(A)')\nplt.ylabel('Transmission')\nplt.title('Cubic Spline Interpolation')\nplt.show()\n"

In [7]:
ws_i = bp_i[0]
bp_i = bp_i[1]
f_bp_i = interp1d(ws_i, bp_i, kind='cubic')
ws_i_new = np.linspace(ws_i[0], ws_i[-1], 1000)
bp_i_new = f_bp_i(ws_i_new)

In [8]:
'''
# Plot the original data points
plt.plot(ws_i, bp_i,label='Data Points',color = 'black')

# Plot the cubic spline
plt.plot(ws_i_new, bp_i_new, label='Cubic Spline', color='red')

plt.legend()
plt.xlabel('wavelength(A)')
plt.ylabel('Transmission')
plt.title('Cubic Spline Interpolation')
plt.show()
'''

"\n# Plot the original data points\nplt.plot(ws_i, bp_i,label='Data Points',color = 'black')\n\n# Plot the cubic spline\nplt.plot(ws_i_new, bp_i_new, label='Cubic Spline', color='red')\n\nplt.legend()\nplt.xlabel('wavelength(A)')\nplt.ylabel('Transmission')\nplt.title('Cubic Spline Interpolation')\nplt.show()\n"

In [9]:
# Define your TARGETID and output file name here
tid = 39627533204853360  # Replace with your TARGETID
outpng = 'output.png'  # Replace with your desired output file name

# Directory where the FITS files are located
# Replace 'PSCRATCH' with the actual path if the environment variable is not set
ardir = os.path.join(os.getenv("PSCRATCH", "/pscratch/sd/z/zhaozhon"))

def get_ws(ardir):
    fn = sorted(
        glob(os.path.join(ardir, "pernight-spectra","desi-ztf-qso-iron-*-*.fits"))
    )[0]
    ws = fitsio.read(fn, "BRZ_WAVE")
    return ws
def get_tid_tileids_nights(tid, all_tids, all_tileids, all_nights):
    sel = all_tids == tid
    return all_tileids[sel], all_nights[sel]
def get_indiv_spectra(tid, tileids, nights, ardir, nwave):

    nobs = len(tileids)

    # read the spectra
    fs = np.zeros((nobs, nwave))  # flux
    ivs = np.zeros((nobs, nwave))  # inverse variance

    # loop on (tileids, nights)
    for i, (tileid, night) in enumerate(zip(tileids, nights)):

        fn = os.path.join(
            ardir,"pernight-spectra",
            "desi-ztf-qso-iron-{}-{}.fits".format(tileid, night),
        )

        # first get the row corresponding to TARGETID
        # (for a given TILEID, a TARGETID can appear only once max.)
        tmp_fm = fitsio.read(fn, "FIBERMAP", columns=["TARGETID"])
        tmp_i = np.where(tmp_fm["TARGETID"] == tid)[0][0]

        # now read the flux, ivar only for that row

        # this is more intuitive, but it loads the whole image first,
        #   then extracts the relevant row, so it s ~slow
        fs[i, :] = fitsio.read(fn, "BRZ_FLUX")[tmp_i, :]
        ivs[i, :] = fitsio.read(fn, "BRZ_IVAR")[tmp_i, :]

        # this way is *much* faster, with reading only the relevant row
        #   note: fitsio wants/works with a slice, not an integer...
        #h = fitsio.FITS(fn)
        #tmp_slice = slice(tmp_i, tmp_i + 1, 1)
        #fs[i, :] = h["BRZ_FLUX"][tmp_slice, :]
        #ivs[i, :] = h["BRZ_IVAR"][tmp_slice, :]

    return fs, ivs
def get_smooth(fs, ivs, gauss_smooth):
    tmp0fs = fs.copy()
    tmp0fs[ivs == 0] = 0
    tmp1fs = 1 + 0 * fs.copy()
    tmp1fs[ivs == 0] = 0
    tmp0smfs = gaussian_filter1d(tmp0fs, gauss_smooth, mode="constant", cval=0)
    tmp1smfs = gaussian_filter1d(tmp1fs, gauss_smooth, mode="constant", cval=0)
    return tmp0smfs / tmp1smfs
def plot_tid_spectra(outpng, tid, ws, fs, ivs):
    fig, ax = plt.subplots(figsize=(10, 5))
    for i in range(fs.shape[0]):
        smfs_i = get_smooth(fs[i], ivs[i], 5)
        ax.plot(ws, smfs_i, lw=0.5)
    ax.set_title("TARGETID = {}".format(tid))
    ax.set_xlabel("Observed wavelength [A]")
    ax.set_ylabel("Flux [erg / cm2 / s / A]")
    ax.grid()
    plt.savefig(outpng, bbox_inches="tight")
    plt.close()
sumfn = os.path.join(ardir, "desi-ztf-qso-iron-pernight-summary.fits")
def save_wavelength_and_flux_to_npy(wavelength, flux, filename1,filename2):
    np.save(filename1, wavelength)
    np.save(filename2,flux)
def k_correction(ws,fs,ws_bp,bp,z0,z1,x_l,x_r):
    fs_avg = np.mean(fs, axis=0)
    f = interp1d(ws,fs_avg, kind='cubic')
    f_bp = interp1d(ws_bp, bp, kind='cubic')
    #area0
    new_ws = np.linspace(ws_bp[0],ws_bp[-1],10000)
    new_fs = f(new_ws)
    new_bp = f_bp(new_ws)
    y_0 = new_ws*new_fs*new_bp
    area0 = np.trapz(y_0,new_ws)
    #area1
    new_ws1 = np.linspace(ws_bp[0]/(1+z0),ws_bp[-1]/(1+z0),10000)
    new_fs1 = f(new_ws1*(1+z1))
    y_1 = new_ws1*new_fs1*new_bp
    area1 = np.trapz(y_1,new_ws1)
    k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
    #if 2.1<z1<2.9:       
        #plt.plot(new_ws1,new_fs1)
        #plt.plot(new_ws1,new_bp)
        #plt.show()
    #zero_point
    #area2 = np.trapz(f_z(new_ws)*new_ws*new_bp,new_ws)
    #area3 = np.trapz(f_z(new_ws1*(1+z1))*new_ws1*new_bp,new_ws1)
    #k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)+2.5*np.log10(area2/area3)
    return k
# wavelengths
ws = get_ws(ardir)
nwave = len(ws)

# read the master table
start = time()
d = fitsio.read(sumfn, "FIBERMAP")
print("reading {} done (took {:.1f}s)".format(sumfn, time() - start))

# get the (TILEID,NIGHT) sets for that TARGETID
tileids, nights = get_tid_tileids_nights(tid, d["TARGETID"], d["TILEID"], d["NIGHT"])

# get fluxes and ivars
start = time()
fs, ivs = get_indiv_spectra(tid, tileids, nights, ardir, nwave)
print("done extracting {} spectra for TARGETID={} (took {:.1f}s)".format(len(tileids), tid, time() - start))

# plot
#plot_tid_spectra(outpng, tid, ws, fs, ivs)
#print("done plotting {}".format(outpng))

reading /pscratch/sd/z/zhaozhon/desi-ztf-qso-iron-pernight-summary.fits done (took 3.2s)
done extracting 4 spectra for TARGETID=39627533204853360 (took 0.7s)


In [10]:
print(np.min(ws))
print(np.max(ws))

3600.0
9824.000000000524


In [11]:

from tqdm import tqdm
k_g = []
tid_g = []
index_g = []
for index,targetid in tqdm(enumerate(targetid_abs_g),total = 900000):
    z_1 = z_g[index]
    if z_1 > 2.1 and z_1 < 2.9:
        ws = get_ws(ardir)
        nwave = len(ws)
        if targetid in d["TARGETID"]:
            tileids, nights = get_tid_tileids_nights(targetid, d["TARGETID"], d["TILEID"], d["NIGHT"])
            fs, ivs = get_indiv_spectra(targetid, tileids, nights, ardir, nwave)
            k = k_correction(ws,fs,ws_g,bp_g,2.3,z_1,ws_g[0],ws_g[-1])
            k_g.append(k)
            tid_g.append(targetid)
            index_g.append(index)
        else:
            continue
    else:
        continue
    #tileids, nights = get_tid_tileids_nights(targetid, d["TARGETID"], d["TILEID"], d["NIGHT"])
    #print(tileids)
    #fs, ivs = get_indiv_spectra(targetid, tileids, nights, ardir, nwave)
    #k = k_correction(ws,fs,2.1,2.3,4000,5500)
    #k_g.append(k)


  0%|          | 1734/900000 [01:04<17:31:59, 14.23it/s]/tmp/ipykernel_1476688/896916668.py:92: RuntimeWarning: divide by zero encountered in log10
  k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
  1%|          | 8152/900000 [05:48<19:21:26, 12.80it/s]/tmp/ipykernel_1476688/896916668.py:92: RuntimeWarning: invalid value encountered in double_scalars
  k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
 13%|█▎        | 119836/900000 [1:27:18<9:28:26, 22.87it/s] 


In [12]:
np.save('k_g',k_g)
np.save('tid_g',tid_g)
np.save('index_g',index_g)

In [13]:
from tqdm import tqdm
k_r = []
tid_r = []
index_r = []
for index,targetid in tqdm(enumerate(targetid_abs_r),total = 100000):
    z_1 = z_r[index]
    if z_1 > 2.1 and z_1 < 2.8:
        ws = get_ws(ardir)
        nwave = len(ws)
        if targetid in d["TARGETID"]:
            tileids, nights = get_tid_tileids_nights(targetid, d["TARGETID"], d["TILEID"], d["NIGHT"])
            fs, ivs = get_indiv_spectra(targetid, tileids, nights, ardir, nwave)
            k = k_correction(ws,fs,ws_r,bp_r,2.5,z_1,ws_r[0],ws_r[-1])
            k_r.append(k)
            tid_r.append(targetid)
            index_r.append(index)
        else:
            continue
    else:
        continue

  3%|▎         | 3003/100000 [02:19<2:00:39, 13.40it/s]/tmp/ipykernel_1476688/896916668.py:92: RuntimeWarning: invalid value encountered in log10
  k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
  9%|▊         | 8599/100000 [06:32<1:57:58, 12.91it/s]/tmp/ipykernel_1476688/896916668.py:92: RuntimeWarning: invalid value encountered in double_scalars
  k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
137580it [1:33:36, 24.50it/s]                           


In [14]:

np.save('k_r',k_r)
np.save('tid_r',tid_r)
np.save('index_r',index_r)


In [15]:
from tqdm import tqdm
k_i = []
tid_i = []
index_i = []
for index,targetid in tqdm(enumerate(targetid_abs_i),total = 100000):
    z_1 = z_i[index]
    if z_1 > 2.1 and z_1 < 2.8:
        ws = get_ws(ardir)
        nwave = len(ws)
        if targetid in d["TARGETID"]:
            tileids, nights = get_tid_tileids_nights(targetid, d["TARGETID"], d["TILEID"], d["NIGHT"])
            fs, ivs = get_indiv_spectra(targetid, tileids, nights, ardir, nwave)
            k = k_correction(ws,fs,ws_i,bp_i,2.5,z_1,ws_i[0],ws_i[-1])
            k_i.append(k)
            tid_i.append(targetid)
            index_i.append(index)
        else:
            continue
    else:
        continue

  8%|▊         | 7536/100000 [04:25<2:08:45, 11.97it/s]/tmp/ipykernel_1476688/896916668.py:92: RuntimeWarning: invalid value encountered in double_scalars
  k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
 62%|██████▏   | 61897/100000 [39:52<13:16, 47.86it/s]  /tmp/ipykernel_1476688/896916668.py:92: RuntimeWarning: invalid value encountered in log10
  k = 2.5*np.log10(1+z1)+2.5*np.log10(area1/area0)
 88%|████████▊ | 87811/100000 [52:55<07:20, 27.66it/s] 


In [16]:

np.save('k_i',k_i)
np.save('tid_i',tid_i)
np.save('index_i',index_i)
